# Filter Detect-seq off-target editing signals

## Load packages

In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
from pandarallel import pandarallel


pd.set_option("max_colwidth", 35)  # column最大宽度
pd.set_option("display.width", 200)  # dataframe宽度
pd.set_option("display.max_columns", None)  # column最大显示数
pd.set_option("display.max_rows", 50)  # row最大显示数
pandarallel.initialize()  # 多线程设置，默认使用全部核心 nb_workers=24

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# 测试 call 点
搞一个老的 TAS-independent 的 mpmat 文件

generate old TAS-independent off-target mpmat to call regions

In [2]:
df_old_final_list = pd.read_csv(
    "./lib/20220312-DdCBE-off_target_type.FinallistV4.CheckPrimer.AddV4ID.tsv",
    sep='\t',
    header=0,
    usecols=['region_id', 'off_target_id.V4.ND4', 'off_target_id.V4.ND5.1', 'off_target_id.V4.ND6'],
    engine='pyarrow'
)
df_old_final_list.columns = ['mpmat_index', 'id_ND4', 'id_ND5.1', 'id_ND6']
df_old_final_list.head()

,mpmat_index,id_ND4,id_ND5.1,id_ND6
0,chr1_1471366_1471410,NA,NA,NA
1,chr1_1693068_1693084,NA,NA,NA
2,chr1_2044988_2044998,NA,ND5.1-TAS.IND-1,NA
3,chr1_5806517_5806533,NA,NA,NA
4,chr1_5983069_5983085,NA,NA,NA


In [3]:
def query_ind(x):
    x = x.fillna('')
    if 'IND' in x['id_ND4']:
        return True
    elif 'IND' in x['id_ND5.1']:
        return True
    elif 'IND' in x['id_ND6']:
        return True
    else:
        return False


df_old_final_list.apply(query_ind, axis=1).sum()

650

In [4]:
df_old_share = df_old_final_list[df_old_final_list.apply(query_ind, axis=1)].reset_index(drop=True)
df_old_share

,mpmat_index,id_ND4,id_ND5.1,id_ND6
0,chr1_2044988_2044998,NA,ND5.1-TAS.IND-1,NA
1,chr1_9702414_9702449,NA,ND5.1-TAS.IND-2,ND6-TAS.IND-1
2,chr1_12618165_12618174,NA,ND5.1-TAS.IND-3,NA
3,chr1_13019998_13020019,NA,NA,ND6-TAS.IND-2
4,chr1_13354705_13354738,NA,NA,ND6-TAS.IND-3
...,...,...,...,...
645,chrX_136925330_136925338,NA,NA,ND6-TAS.IND-538
646,chrX_138128211_138128237,NA,NA,ND6-TAS.IND-539
647,chrX_150683735_150683758,NA,ND5.1-TAS.IND-452,ND6-TAS.IND-540
648,chrX_153669436_153669453,NA,ND5.1-TAS.IND-453,ND6-TAS.IND-541


In [5]:
# 拿 650 个 old share 的点做一个 mpmat 出来去对着这五个 sample call 一下相关位置
df_mpmat = pd.read_csv(
    '../tables/293T-DdCBE-merge_hg38.MAPQ20.C6_M4_R1_T10.sort.V4.mpmat.gz',
    sep='\t',
    header=None,
)
df_mpmat['mpmat_index'] = df_mpmat[0] + '_' + df_mpmat[1].astype(str) + '_' + df_mpmat[2].astype(str)
df_mpmat

,0,1,2,3,4,5,6,7,8,9,10,11,12,mpmat_index
0,chr1,48137,48144,3,2,0,"chr1_48137_CT,chr1_48139_CT,chr...","4,2,4","19,19,19","0.21053,0.10526,0.21053","False,False,False","0,0,0","Pass,Filter,Pass",chr1_48137_48144
1,chr1,127550,127558,2,2,0,"chr1_127550_CT,chr1_127558_CT","5,5","22,21","0.22727,0.2381","False,False","0,0","Pass,Pass",chr1_127550_127558
2,chr1,261533,261543,2,2,0,"chr1_261533_GA,chr1_261543_GA","4,4","14,17","0.28571,0.23529","False,False","0,0","Pass,Pass",chr1_261533_261543
3,chr1,629090,629115,9,4,0,"chr1_629090_CT,chr1_629091_CT,c...","27,3,5,23,14,8,24,2,90","716,807,1057,1448,1633,1884,209...","0.03771,0.00372,0.00473,0.01588...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0","Pass,Filter,Filter,Pass,Filter,...",chr1_629090_629115
4,chr1,629136,629148,6,2,0,"chr1_629136_CT,chr1_629138_CT,c...","100,10,19,0,9,68","3579,3571,3591,3604,3614,3617","0.02794,0.0028,0.00529,0.0,0.00...","False,False,False,False,False,F...","0,0,0,0,0,0","Pass,Filter,Filter,Filter,Filte...",chr1_629136_629148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103310,chrM,16218,16261,20,18,1,"chrM_16218_CT,chrM_16221_CT,chr...","142,102,109,61,179,94,147,120,1...","7817,7756,7847,7891,7860,7832,7...","0.01817,0.01315,0.01389,0.00773...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...","Pass,Pass,Pass,Filter,Pass,Pass...",chrM_16218_16261
103311,chrM,16255,16274,3,3,0,"chrM_16255_GA,chrM_16273_GA,chr...","110,145,337","7536,7620,7678","0.0146,0.01903,0.04389","False,False,False","0,0,0","Pass,Pass,Pass",chrM_16255_16274
103312,chrM,16259,16279,11,8,1,"chrM_16259_CT,chrM_16260_CT,chr...","104,85,87,58,129,107,93,103,712...","7715,7730,7801,7811,7836,7758,7...","0.01348,0.011,0.01115,0.00743,0...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0,0,0","Pass,Pass,Pass,Filter,Pass,Pass...",chrM_16259_16279
103313,chrM,16329,16373,7,7,0,"chrM_16329_GA,chrM_16336_GA,chr...","131,147,246,421,466,656,106","7673,7755,7746,7663,7797,7764,7859","0.01707,0.01896,0.03176,0.05494...","False,False,False,False,False,F...","0,0,0,0,0,0,0","Pass,Pass,Pass,Pass,Pass,Pass,Pass",chrM_16329_16373


In [6]:
df_merge = pd.merge(left=df_mpmat, right=df_old_share, on='mpmat_index', how='right')

df_aim_mpmat = df_merge.iloc[:, :-4]
df_aim_mpmat

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,chr1,2044988,2044998,7,6,0,"chr1_2044988_CT,chr1_2044989_CT...","14,11,7,11,0,8,5","25,25,25,23,22,22,23","0.56,0.44,0.28,0.47826,0.0,0.36...","False,False,False,False,False,F...","0,0,0,0,0,0,0","Pass,Pass,Pass,Pass,Filter,Pass..."
1,chr1,9702414,9702449,11,9,0,"chr1_9702414_GA,chr1_9702415_GA...","4,1,3,8,7,7,18,10,20,12,32","103,100,96,100,94,97,91,91,93,9...","0.03883,0.01,0.03125,0.08,0.074...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0,0,0","Pass,Filter,Filter,Pass,Pass,Pa..."
2,chr1,12618165,12618174,5,5,0,"chr1_12618165_CT,chr1_12618166_...","11,4,11,8,7","26,29,29,31,26","0.42308,0.13793,0.37931,0.25806...","False,False,False,False,False","0,0,0,0,0","Pass,Pass,Pass,Pass,Pass"
3,chr1,13019998,13020019,9,8,0,"chr1_13019998_GA,chr1_13020006_...","7,2,7,5,5,11,7,7,19","38,42,40,42,42,41,42,41,42","0.18421,0.04762,0.175,0.11905,0...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0","Pass,Filter,Pass,Pass,Pass,Pass..."
4,chr1,13354705,13354738,11,11,0,"chr1_13354705_CT,chr1_13354706_...","37,9,28,23,13,16,12,4,10,4,5","63,61,66,68,67,68,65,68,68,71,69","0.5873,0.14754,0.42424,0.33824,...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0,0,0","Pass,Pass,Pass,Pass,Pass,Pass,P..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,chrX,136925330,136925338,5,5,0,"chrX_136925330_CT,chrX_13692533...","10,6,15,8,9","31,31,30,32,31","0.32258,0.19355,0.5,0.25,0.29032","False,False,False,False,False","0,0,0,0,0","Pass,Pass,Pass,Pass,Pass"
646,chrX,138128211,138128237,9,7,0,"chrX_138128211_GA,chrX_13812821...","5,4,3,4,4,3,16,9,26","37,38,38,38,38,34,37,37,35","0.13514,0.10526,0.07895,0.10526...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0","Pass,Pass,Filter,Pass,Pass,Filt..."
647,chrX,150683735,150683758,11,11,0,"chrX_150683735_GA,chrX_15068373...","8,12,13,15,35,41,75,57,52,90,9","217,211,211,209,207,204,211,203...","0.03687,0.05687,0.06161,0.07177...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0,0,0","Pass,Pass,Pass,Pass,Pass,Pass,P..."
648,chrX,153669436,153669453,10,8,0,"chrX_153669436_CT,chrX_15366943...","28,22,25,11,4,7,3,7,3,7","62,64,61,66,64,66,67,66,65,67","0.45161,0.34375,0.40984,0.16667...","False,False,False,False,False,F...","0,0,0,0,0,0,0,0,0,0","Pass,Pass,Pass,Pass,Pass,Pass,F..."


In [7]:
# os.makedirs('../mpmat', exist_ok=True)
# df_aim_mpmat.to_csv('../mpmat/2022-10-21_nature_4_5.1_6_share_650-off-targets.mpmat', header=None, index=None, sep='\t')

## analysis with old TAS-independent list (650 sites) -> df_aim_mpmat

### Detect-seq signal comparation for DddA\_wt/6/11

In [93]:
# # 650 mpmat to get 650 poisson_res
# df_old_site_new_signal = pd.read_csv(
#     '../poisson_res_use_old650/poisson_res_all_use650.tsv.gz',
#     header=0,
#     index_col=None,
#     sep='\t',
#     engine='pyarrow'
# )
# df_old_site_new_signal

In [94]:
# df_old_site_new_signal.info()

In [95]:
# df_old_site_new_signal.isna().sum().sum()

In [96]:
# df_old_site_new_signal = df_old_site_new_signal.assign(
#     bed_name=df_old_site_new_signal.mpmat_index + '_highest_' + df_old_site_new_signal.region_highest_site_index,
#     strand='.'
# )
# df_old_site_new_signal

In [97]:
# df_sign_strict = (df_old_site_new_signal
#                   .query('FDR <= 0.0001')  # 10911
#                   .query('log2_FC_mut >= 2')  # 10911
#                   .query('ctrl_mut_count <= 1')  # 9551
#                   .query('`treat_mut_count.norm` * 100 >= 10')  # 9551
#                   .query('treat_mut_count >= 20')  # 9551
#                   .query('treat_mut_count / treat_count >= 0.15')  # 6080
#                   .query('region_block_site_num <= 1')  # 5720
#                   .query('region_highest_site_mut_ratio >= 0.35')  # 2929
#                   )

# print(df_sign_strict.shape[0])
# df_sign_strict.groupby('<sample>').describe()

In [98]:
# df_sign_lenient = (df_old_site_new_signal
#                    .query('FDR <= 0.01')  # 25118
#                    .query('log2_FC_mut >= 2')  # 25118
#                    .query('ctrl_mut_count <= 1')  # 22009
#                    .query('`treat_mut_count.norm` * 100 >= 5')  # 22009
#                    .query('treat_mut_count >= 10')  # 22009
#                    .query('treat_mut_count / treat_count >= 0.15')  # 14476
#                    .query('region_block_site_num <= 1')  # 13522
#                    .query('region_highest_site_mut_ratio >= 0.30')  # 7001
#                    )

# print(df_sign_lenient.shape[0])
# df_sign_lenient.groupby('<sample>').describe()

In [99]:
# df_sign_lenient

In [100]:
# df_sign_lenient.mpmat_index.describe()

In [103]:
# df_old_439 = df_sign_lenient

### data processing

In [101]:
# df_old_439 = df_old_439[['<sample>', 'mpmat_index',
#                          'treat_mut_count.norm', 'treat_count.norm']].copy()
# df_old_439 = (
#     df_old_439.
#     assign(
#         sample=df_old_439['<sample>'],
#         detect_seq_score=df_old_439.apply(
#             lambda x: ((x['treat_mut_count.norm'] / x['treat_count.norm']) ** 2) * x['treat_mut_count.norm'], axis=1),
#     )
# )

# df_old_439

In [102]:
# df_detect_seq_score = df_old_439[['mpmat_index', 'sample', 'detect_seq_score']].copy()
# df_share_old = df_detect_seq_score[['mpmat_index', 'sample', 'detect_seq_score']].copy()
# df_share_old

# 初步过滤 poisson_res -> df_pois 并保存备用
- log2_FC: log2(treat_count.norm/ctrl_count.norm), 如果ctrl_count.norm不存在，就用 chr 突变背景
- log2_FC_mut: log2(treat_mut_count.norm/ctrl_mut_count.norm), 如果ctrl_mut_count.norm不存在，就用 chr 突变背景
- region_block_state: B-Blocked, S-SNV, N-Non-SNV

## 使用 cutoff 筛选 off-target

In [8]:
file = '../poisson_res_ok_bak/poisson_res_all.tsv.gz'

In [9]:
# %%time
# 大量耗时！
df = pd.read_csv(
    file,
    header=0,
    index_col=None,
    sep='\t',
    engine='pyarrow'
)
df.head()

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_site_index,region_block_state,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
0,DetectSeq_ATP8-DddA11_REP-1,chr1,16395,16418,chr1_16395_16418,11,0,11,"chr1_16395_CT,chr1_16397_C.,chr...",N-N-N-N-N-N-N-N-N-N-N,chr1_16395_CT,3,16,0.187500,57,19,0,2,0.144296,0.113256,0.0,0.011922,"0,1,2,3,4,5,6,7,8,9,10,11 53,4,...",-0.349446,0.557555,TestOK,0.377620,0.842234
1,DetectSeq_ATP8-DddA11_REP-1,chr1,16534,16540,chr1_16534_16540,4,1,3,"chr1_16534_CT,chr1_16538_CT,chr...",B-N-N-N,chr1_16538_CT,0,7,0.000000,36,8,0,0,0.091135,0.047687,0.0,0.000000,"0,1,2,3,4 36,0,0,0,0 8,0,0,0,0",-0.934408,NaN,TestOK,0.841752,0.842234
2,DetectSeq_ATP8-DddA11_REP-1,chr1,20272,20284,chr1_20272_20284,5,0,5,"chr1_20272_GA,chr1_20275_GA,chr...",N-N-N-N-N,chr1_20283_GA,2,7,0.285714,12,9,0,2,0.030378,0.053648,0.0,0.011922,"0,1,2,3,4,5 11,1,0,0,0,0 5,2,0,...",0.820479,0.557555,TestOK,0.377620,0.842234
3,DetectSeq_ATP8-DddA11_REP-1,chr1,31029,31037,chr1_31029_31037,4,1,3,"chr1_31029_GA,chr1_31030_G.,chr...",B-N-N-N,chr1_31031_GA,1,6,0.166667,2,6,0,1,0.005063,0.035765,0.0,0.005961,"0,1,2,3,4 2,0,0,0,0 5,0,1,0,0",2.820479,-0.442445,TestOK,0.582932,0.842234
4,DetectSeq_ATP8-DddA11_REP-1,chr1,54043,54047,chr1_54043_54047,3,0,3,"chr1_54043_CT,chr1_54044_C.,chr...",N-N-N,chr1_54043_CT,4,4,1.000000,0,4,0,0,0.000000,0.023843,0.0,0.000000,"0,1,2,3 0,0,0,0 0,4,0,0",-1.456561,NaN,TestOK,0.841752,0.842234


In [10]:
df.columns

Index(['<sample>', 'chr_name', 'region_start', 'region_end', 'mpmat_index', 'region_site_num', 'region_block_site_num', 'region_mut_site_num', 'region_site_index', 'region_block_state',
       'region_highest_site_index', 'region_highest_site_mut_num', 'region_highest_site_cover_num', 'region_highest_site_mut_ratio', 'ctrl_count', 'treat_count', 'ctrl_mut_count', 'treat_mut_count',
       'ctrl_count.norm', 'treat_count.norm', 'ctrl_mut_count.norm', 'treat_mut_count.norm', 'count_info', 'log2_FC', 'log2_FC_mut', 'test_state', 'p_value', 'FDR'],
      dtype='object')

In [11]:
df.drop(columns=['region_site_index', 'region_block_state'], inplace=True)

In [12]:
# print(df.info())
print(df.isna().sum())
# NAN应该是 test sample 中的，一会过滤一下

<sample>                               0
chr_name                               0
region_start                           0
region_end                             0
mpmat_index                            0
region_site_num                        0
region_block_site_num                  0
region_mut_site_num                    0
region_highest_site_index              0
region_highest_site_mut_num            0
region_highest_site_cover_num          0
region_highest_site_mut_ratio          0
ctrl_count                             0
treat_count                            0
ctrl_mut_count                         0
treat_mut_count                        0
ctrl_count.norm                        0
treat_count.norm                       0
ctrl_mut_count.norm                    0
treat_mut_count.norm                   0
count_info                             0
log2_FC                          1545839
log2_FC_mut                      8948384
test_state                             0
p_value         

In [13]:
print(df['<sample>'].unique())

['DetectSeq_ATP8-DddA11_REP-1' 'DetectSeq_ATP8-DddA6_REP-1'
 'DetectSeq_ATP8-DddAwt_REP-1' 'DetectSeq_JAK2-DddA11_REP-1'
 'DetectSeq_JAK2-DddA11_REP-2' 'DetectSeq_SIRT6-DddA11_REP-1'
 'DetectSeq_SIRT6-DddA11_REP-2' 'test']


In [14]:
df = df.assign(
    bed_name=df.mpmat_index + '_highest_' + df.region_highest_site_index,
    strand='.'
)
df = df[df['<sample>'] != 'test']
df

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR,bed_name,strand
0,DetectSeq_ATP8-DddA11_REP-1,chr1,16395,16418,chr1_16395_16418,11,0,11,chr1_16395_CT,3,16,0.187500,57,19,0,2,0.144296,0.113256,0.0,0.011922,"0,1,2,3,4,5,6,7,8,9,10,11 53,4,...",-0.349446,0.557555,TestOK,0.377620,0.842234,chr1_16395_16418_highest_chr1_1...,.
1,DetectSeq_ATP8-DddA11_REP-1,chr1,16534,16540,chr1_16534_16540,4,1,3,chr1_16538_CT,0,7,0.000000,36,8,0,0,0.091135,0.047687,0.0,0.000000,"0,1,2,3,4 36,0,0,0,0 8,0,0,0,0",-0.934408,NaN,TestOK,0.841752,0.842234,chr1_16534_16540_highest_chr1_1...,.
2,DetectSeq_ATP8-DddA11_REP-1,chr1,20272,20284,chr1_20272_20284,5,0,5,chr1_20283_GA,2,7,0.285714,12,9,0,2,0.030378,0.053648,0.0,0.011922,"0,1,2,3,4,5 11,1,0,0,0,0 5,2,0,...",0.820479,0.557555,TestOK,0.377620,0.842234,chr1_20272_20284_highest_chr1_2...,.
3,DetectSeq_ATP8-DddA11_REP-1,chr1,31029,31037,chr1_31029_31037,4,1,3,chr1_31031_GA,1,6,0.166667,2,6,0,1,0.005063,0.035765,0.0,0.005961,"0,1,2,3,4 2,0,0,0,0 5,0,1,0,0",2.820479,-0.442445,TestOK,0.582932,0.842234,chr1_31029_31037_highest_chr1_3...,.
4,DetectSeq_ATP8-DddA11_REP-1,chr1,54043,54047,chr1_54043_54047,3,0,3,chr1_54043_CT,4,4,1.000000,0,4,0,0,0.000000,0.023843,0.0,0.000000,"0,1,2,3 0,0,0,0 0,4,0,0",-1.456561,NaN,TestOK,0.841752,0.842234,chr1_54043_54047_highest_chr1_5...,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10744925,DetectSeq_SIRT6-DddA11_REP-2,chrX,155984850,155984857,chrX_155984850_155984857,2,0,2,chrX_155984850_CT,4,6,0.666667,0,7,0,0,0.000000,0.032510,0.0,0.000000,"0,1,2 0,0,0 2,5,0",-0.830575,NaN,TestOK,0.838398,0.897853,chrX_155984850_155984857_highes...,.
10744926,DetectSeq_SIRT6-DddA11_REP-2,chrX,155984970,155984999,chrX_155984970_155984999,7,0,7,chrX_155984970_CT,0,8,0.000000,0,10,0,0,0.000000,0.046442,0.0,0.000000,"0,1,2,3,4,5,6,7 0,0,0,0,0,0,0,0...",-0.316002,NaN,TestOK,0.838398,0.897853,chrX_155984970_155984999_highes...,.
10744927,DetectSeq_SIRT6-DddA11_REP-2,chrX,155985095,155985105,chrX_155985095_155985105,3,0,3,chrX_155985095_GA,4,4,1.000000,0,4,0,2,0.000000,0.018577,0.0,0.009288,"0,1,2,3 0,0,0,0 1,1,2,0",-1.637930,0.465867,TestOK,0.372391,0.897853,chrX_155985095_155985105_highes...,.
10744928,DetectSeq_SIRT6-DddA11_REP-2,chrX,155988525,155988531,chrX_155988525_155988531,4,0,4,chrX_155988525_CT,0,14,0.000000,0,15,0,0,0.000000,0.069663,0.0,0.000000,"0,1,2,3,4 0,0,0,0,0 15,0,0,0,0",0.268960,NaN,TestOK,0.838398,0.897853,chrX_155988525_155988531_highes...,.


In [15]:
# print(df.info())
print(df.isna().sum())
# 过滤掉 test sample 之后 NAN 少了很多

<sample>                               0
chr_name                               0
region_start                           0
region_end                             0
mpmat_index                            0
region_site_num                        0
region_block_site_num                  0
region_mut_site_num                    0
region_highest_site_index              0
region_highest_site_mut_num            0
region_highest_site_cover_num          0
region_highest_site_mut_ratio          0
ctrl_count                             0
treat_count                            0
ctrl_mut_count                         0
treat_mut_count                        0
ctrl_count.norm                        0
treat_count.norm                       0
ctrl_mut_count.norm                    0
treat_mut_count.norm                   0
count_info                             0
log2_FC                            13944
log2_FC_mut                      7413509
test_state                             0
p_value         

In [16]:
# 这里mpmat_index unique，即 frequency == 1 才合理，因为前期 call 点的时候，把所有的 region 放在一起考虑的
df.groupby('<sample>').mpmat_index.describe()
# call 点没问题
# 继续后续分析

,count,unique,top,freq
<sample>,,,,
DetectSeq_ATP8-DddA11_REP-1,1534990,1534990,chr1_16395_16418,1
DetectSeq_ATP8-DddA6_REP-1,1534990,1534990,chr1_16395_16418,1
DetectSeq_ATP8-DddAwt_REP-1,1534990,1534990,chr1_16395_16418,1
DetectSeq_JAK2-DddA11_REP-1,1534990,1534990,chr1_16395_16418,1
DetectSeq_JAK2-DddA11_REP-2,1534990,1534990,chr1_16395_16418,1
DetectSeq_SIRT6-DddA11_REP-1,1534990,1534990,chr1_16395_16418,1
DetectSeq_SIRT6-DddA11_REP-2,1534990,1534990,chr1_16395_16418,1


## find significant region

In [17]:
df

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR,bed_name,strand
0,DetectSeq_ATP8-DddA11_REP-1,chr1,16395,16418,chr1_16395_16418,11,0,11,chr1_16395_CT,3,16,0.187500,57,19,0,2,0.144296,0.113256,0.0,0.011922,"0,1,2,3,4,5,6,7,8,9,10,11 53,4,...",-0.349446,0.557555,TestOK,0.377620,0.842234,chr1_16395_16418_highest_chr1_1...,.
1,DetectSeq_ATP8-DddA11_REP-1,chr1,16534,16540,chr1_16534_16540,4,1,3,chr1_16538_CT,0,7,0.000000,36,8,0,0,0.091135,0.047687,0.0,0.000000,"0,1,2,3,4 36,0,0,0,0 8,0,0,0,0",-0.934408,NaN,TestOK,0.841752,0.842234,chr1_16534_16540_highest_chr1_1...,.
2,DetectSeq_ATP8-DddA11_REP-1,chr1,20272,20284,chr1_20272_20284,5,0,5,chr1_20283_GA,2,7,0.285714,12,9,0,2,0.030378,0.053648,0.0,0.011922,"0,1,2,3,4,5 11,1,0,0,0,0 5,2,0,...",0.820479,0.557555,TestOK,0.377620,0.842234,chr1_20272_20284_highest_chr1_2...,.
3,DetectSeq_ATP8-DddA11_REP-1,chr1,31029,31037,chr1_31029_31037,4,1,3,chr1_31031_GA,1,6,0.166667,2,6,0,1,0.005063,0.035765,0.0,0.005961,"0,1,2,3,4 2,0,0,0,0 5,0,1,0,0",2.820479,-0.442445,TestOK,0.582932,0.842234,chr1_31029_31037_highest_chr1_3...,.
4,DetectSeq_ATP8-DddA11_REP-1,chr1,54043,54047,chr1_54043_54047,3,0,3,chr1_54043_CT,4,4,1.000000,0,4,0,0,0.000000,0.023843,0.0,0.000000,"0,1,2,3 0,0,0,0 0,4,0,0",-1.456561,NaN,TestOK,0.841752,0.842234,chr1_54043_54047_highest_chr1_5...,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10744925,DetectSeq_SIRT6-DddA11_REP-2,chrX,155984850,155984857,chrX_155984850_155984857,2,0,2,chrX_155984850_CT,4,6,0.666667,0,7,0,0,0.000000,0.032510,0.0,0.000000,"0,1,2 0,0,0 2,5,0",-0.830575,NaN,TestOK,0.838398,0.897853,chrX_155984850_155984857_highes...,.
10744926,DetectSeq_SIRT6-DddA11_REP-2,chrX,155984970,155984999,chrX_155984970_155984999,7,0,7,chrX_155984970_CT,0,8,0.000000,0,10,0,0,0.000000,0.046442,0.0,0.000000,"0,1,2,3,4,5,6,7 0,0,0,0,0,0,0,0...",-0.316002,NaN,TestOK,0.838398,0.897853,chrX_155984970_155984999_highes...,.
10744927,DetectSeq_SIRT6-DddA11_REP-2,chrX,155985095,155985105,chrX_155985095_155985105,3,0,3,chrX_155985095_GA,4,4,1.000000,0,4,0,2,0.000000,0.018577,0.0,0.009288,"0,1,2,3 0,0,0,0 1,1,2,0",-1.637930,0.465867,TestOK,0.372391,0.897853,chrX_155985095_155985105_highes...,.
10744928,DetectSeq_SIRT6-DddA11_REP-2,chrX,155988525,155988531,chrX_155988525_155988531,4,0,4,chrX_155988525_CT,0,14,0.000000,0,15,0,0,0.000000,0.069663,0.0,0.000000,"0,1,2,3,4 0,0,0,0,0 15,0,0,0,0",0.268960,NaN,TestOK,0.838398,0.897853,chrX_155988525_155988531_highes...,.


### strict selection 

In [18]:
# df_sign_strict = (
#     df
#     .query('FDR <= 0.0001')
#     .query('log2_FC_mut >= 2')
#     .query('ctrl_mut_count <= 1')
#     .query('`treat_mut_count.norm` * 100 >= 10')
#     .query('treat_mut_count >= 20')
#     .query('treat_mut_count / treat_count >= 0.15')
#     .query('region_block_site_num <= 1')
#     .query('region_highest_site_mut_ratio >= 0.35')  # 1892
# )
#
# print(df_sign_strict.shape[0])
# df_sign_strict.groupby('<sample>').mpmat_index.count()

In [19]:
# df_sign_strict.isna().sum().sum()

### lenient selection

In [20]:
# nature condicitons
# df_sign_lenient = (
#     df
#     .query('FDR <= 0.01')
#     .query('log2_FC_mut >= 2')
#     .query('ctrl_mut_count <= 1')
#     .query('`treat_mut_count.norm` * 100 >= 5')
#     .query('treat_mut_count >= 10')
#     .query('treat_mut_count / treat_count >= 0.15')
#     .query('region_block_site_num <= 1')
#     .query('region_highest_site_mut_ratio >= 0.30')  # 4007
# )


df_sign_lenient = (
    df
    .query('FDR <= 0.01')
    .query('log2_FC_mut >= 2')
    .query('ctrl_mut_count <= 1')
    .query('`treat_mut_count.norm` * 100 >= 5')
    .query('treat_mut_count >= 10')
    .query('treat_mut_count / treat_count >= 0.10')  # TODO
    .query('region_block_site_num <= 1')
    .query('region_highest_site_mut_ratio >= 0.25')  # TODO
)

print(df_sign_lenient.shape[0])
df_sign_lenient.groupby('<sample>').mpmat_index.count()

12349


<sample>
DetectSeq_ATP8-DddA11_REP-1      177
DetectSeq_ATP8-DddA6_REP-1       589
DetectSeq_ATP8-DddAwt_REP-1       57
DetectSeq_JAK2-DddA11_REP-1      628
DetectSeq_JAK2-DddA11_REP-2      667
DetectSeq_SIRT6-DddA11_REP-1    2801
DetectSeq_SIRT6-DddA11_REP-2    7430
Name: mpmat_index, dtype: int64

In [109]:
df_sign_lenient.isna().sum().sum()

0

In [110]:
# df = df_sign_lenient
print(df_sign_lenient.shape)
df_sign_lenient.head()

(12349, 28)


,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR,bed_name,strand
83,DetectSeq_ATP8-DddA11_REP-1,chr1,630967,631005,chr1_630967_631005,7,1,6,chr1_630992_GA,2975,4635,0.641855,38,6608,1,780,0.096198,39.389272,0.002532,4.649460,"0,1,2,3,4,5,6,7 35,2,1,0,0,0,0,...",8.677587,10.842847,TestOK,2.511695e-310,5.507753e-305,chr1_630967_631005_highest_chr1...,.
84,DetectSeq_ATP8-DddA11_REP-1,chr1,631016,631037,chr1_631016_631037,6,0,6,chr1_631036_CT,1732,4662,0.371514,33,6236,0,1189,0.083540,37.171837,0.000000,7.087446,"0,1,2,3,4,5,6 33,0,0,0,0,0,0 31...",8.797528,9.773088,TestOK,0.000000e+00,0.000000e+00,chr1_631016_631037_highest_chr1...,.
89,DetectSeq_ATP8-DddA11_REP-1,chr1,631778,631791,chr1_631778_631791,3,0,3,chr1_631791_CT,733,1147,0.639058,40,1483,0,547,0.101261,8.839935,0.000000,3.260583,"0,1,2,3 40,0,0,0 534,278,413,258",6.447891,8.652952,TestOK,2.915445e-214,4.972421e-209,chr1_631778_631791_highest_chr1...,.
95,DetectSeq_ATP8-DddA11_REP-1,chr1,632382,632387,chr1_632382_632387,2,0,2,chr1_632387_GA,4739,6544,0.724175,25,8609,0,1032,0.063288,51.316925,0.000000,6.151593,"0,1,2 24,1,0 1601,5305,1703",9.663290,9.568782,TestOK,0.000000e+00,0.000000e+00,chr1_632382_632387_highest_chr1...,.
100,DetectSeq_ATP8-DddA11_REP-1,chr1,633067,633147,chr1_633067_633147,11,0,11,chr1_633100_GA,696,1232,0.564935,33,2479,0,572,0.083540,14.776938,0.000000,3.409604,"0,1,2,3,4,5,6,7,8,9,10,11 33,0,...",7.466665,8.717426,TestOK,1.596361e-224,3.062998e-219,chr1_633067_633147_highest_chr1...,.


### 标记是否报告为脱靶位点

### 把 Nature 中的 region 对应过来，使用 Nature 中的 idx, fix mpmat_index (add old into it) -> df_pois

In [54]:
df_old_id = pd.read_csv(
    "./lib/20220312-DdCBE-off_target_type.FinallistV4.CheckPrimer.AddV4ID.tsv",
    sep='\t', header=0, usecols=['region_id'], )
df_old_id.region_id.describe()

# 合理 freq == 1

count                     6881
unique                    6881
top       chr1_1471366_1471410
freq                         1
Name: region_id, dtype: object

In [55]:
df_old_id[['chrom', 'start', 'end'
           ]] = pd.Series(df_old_id.region_id.unique()).str.split('_',
                                                                  expand=True)
df_old_id = df_old_id.iloc[:, 1:4]
df_old_id[['start', 'end']] = df_old_id[['start', 'end']].astype(int)
df_old_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6881 entries, 0 to 6880
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   chrom   6881 non-null   object
 1   start   6881 non-null   int64 
 2   end     6881 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 161.4+ KB


In [56]:
df_old_id.head(2)

,chrom,start,end
0,chr1,1471366,1471410
1,chr1,1693068,1693084


In [57]:
df_new_id = pd.Series(df_sign_lenient.mpmat_index.unique()).str.split('_', expand=True)
df_new_id

,0,1,2
0,chr1,630967,631005
1,chr1,631016,631037
2,chr1,631778,631791
3,chr1,632382,632387
4,chr1,633067,633147
...,...,...,...
9961,chrX,154031921,154031945
9962,chrX,154313332,154313399
9963,chrX,154679199,154679238
9964,chrX,154817582,154817611


In [58]:
df_new_id.columns = ['chrom', 'start', 'end']
df_new_id[['start', 'end']] = df_new_id[['start', 'end']].astype(int)

print(df_new_id.info())
df_new_id.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9966 entries, 0 to 9965
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   chrom   9966 non-null   object
 1   start   9966 non-null   int64 
 2   end     9966 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 233.7+ KB
None


,chrom,start,end
0,chr1,630967,631005
1,chr1,631016,631037


In [59]:
from pybedtools import BedTool


bed_new = BedTool.from_dataframe(df_new_id)
bed_nat = BedTool.from_dataframe(df_old_id)

> https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html

In [60]:
df_bed_to_fix = bed_new.intersect(bed_nat, loj=True).to_dataframe()
df_bed_to_fix.columns = ['chrom', 'start', 'end', 'chrom2', 'start2', 'end2']

df_bed_to_fix

,chrom,start,end,chrom2,start2,end2
0,chr1,630967,631005,.,-1,-1
1,chr1,631016,631037,.,-1,-1
2,chr1,631778,631791,.,-1,-1
3,chr1,632382,632387,.,-1,-1
4,chr1,633067,633147,.,-1,-1
...,...,...,...,...,...,...
9963,chrX,154031921,154031945,.,-1,-1
9964,chrX,154313332,154313399,.,-1,-1
9965,chrX,154679199,154679238,.,-1,-1
9966,chrX,154817582,154817611,.,-1,-1


In [61]:
# 新点中有 434 个点和老点有 overlap
df_bed_to_fix[df_bed_to_fix.end2 != -1]

,chrom,start,end,chrom2,start2,end2
9,chr1,18907073,18907133,chr1,18907111,18907133
11,chr1,29332389,29332405,chr1,29332374,29332405
12,chr1,67532549,67532593,chr1,67532557,67532593
13,chr1,70133031,70133115,chr1,70133094,70133130
14,chr1,78617681,78617723,chr1,78617702,78617723
...,...,...,...,...,...,...
8156,chr12,117022857,117022923,chr12,117022835,117022888
8255,chr13,49219804,49219857,chr13,49219804,49219836
8933,chr16,78770741,78770746,chr16,78770741,78770752
8995,chr17,9576625,9576657,chr17,9576625,9576648


In [62]:
df_old_share

,mpmat_index,id_ND4,id_ND5.1,id_ND6
0,chr1_2044988_2044998,NA,ND5.1-TAS.IND-1,NA
1,chr1_9702414_9702449,NA,ND5.1-TAS.IND-2,ND6-TAS.IND-1
2,chr1_12618165_12618174,NA,ND5.1-TAS.IND-3,NA
3,chr1_13019998_13020019,NA,NA,ND6-TAS.IND-2
4,chr1_13354705_13354738,NA,NA,ND6-TAS.IND-3
...,...,...,...,...
645,chrX_136925330_136925338,NA,NA,ND6-TAS.IND-538
646,chrX_138128211_138128237,NA,NA,ND6-TAS.IND-539
647,chrX_150683735_150683758,NA,ND5.1-TAS.IND-452,ND6-TAS.IND-540
648,chrX_153669436_153669453,NA,ND5.1-TAS.IND-453,ND6-TAS.IND-541


In [63]:
# 看一下这里面有多少个是4、5.1、6 的 IND
# df_old_share.mpmat_index.unique()
tmpdf = df_bed_to_fix[df_bed_to_fix.end2 != -1].iloc[:, -3:].copy()
tmpdf.drop_duplicates(keep='first', inplace=True)
tmpdf

,chrom2,start2,end2
9,chr1,18907111,18907133
11,chr1,29332374,29332405
12,chr1,67532557,67532593
13,chr1,70133094,70133130
14,chr1,78617702,78617723
...,...,...,...
8156,chr12,117022835,117022888
8255,chr13,49219804,49219836
8933,chr16,78770741,78770752
8995,chr17,9576625,9576648


In [64]:
tmpls = tmpdf.parallel_apply(lambda x: f'{x[0]}_{x[1]}_{x[2]}', axis=1).tolist()
tmpls = list(set(tmpls))
print(len(tmpls))

433


In [65]:
len(df_old_share.mpmat_index.unique())

650

In [66]:
print(len([i for i in tmpls if i in df_old_share.mpmat_index.unique()]))

362


In [67]:
# form Old IND456 bed
df_old_share['id_merge'] = df_old_share.iloc[:, 1:4].astype(str).agg('_'.join, axis=1)
df_old_share[list('abc')] = df_old_share['mpmat_index'].str.split('_', expand=True)
df_old_share

,mpmat_index,id_ND4,id_ND5.1,id_ND6,id_merge,a,b,c
0,chr1_2044988_2044998,NA,ND5.1-TAS.IND-1,NA,NA_ND5.1-TAS.IND-1_NA,chr1,2044988,2044998
1,chr1_9702414_9702449,NA,ND5.1-TAS.IND-2,ND6-TAS.IND-1,NA_ND5.1-TAS.IND-2_ND6-TAS.IND-1,chr1,9702414,9702449
2,chr1_12618165_12618174,NA,ND5.1-TAS.IND-3,NA,NA_ND5.1-TAS.IND-3_NA,chr1,12618165,12618174
3,chr1_13019998_13020019,NA,NA,ND6-TAS.IND-2,NA_NA_ND6-TAS.IND-2,chr1,13019998,13020019
4,chr1_13354705_13354738,NA,NA,ND6-TAS.IND-3,NA_NA_ND6-TAS.IND-3,chr1,13354705,13354738
...,...,...,...,...,...,...,...,...
645,chrX_136925330_136925338,NA,NA,ND6-TAS.IND-538,NA_NA_ND6-TAS.IND-538,chrX,136925330,136925338
646,chrX_138128211_138128237,NA,NA,ND6-TAS.IND-539,NA_NA_ND6-TAS.IND-539,chrX,138128211,138128237
647,chrX_150683735_150683758,NA,ND5.1-TAS.IND-452,ND6-TAS.IND-540,NA_ND5.1-TAS.IND-452_ND6-TAS.IN...,chrX,150683735,150683758
648,chrX_153669436_153669453,NA,ND5.1-TAS.IND-453,ND6-TAS.IND-541,NA_ND5.1-TAS.IND-453_ND6-TAS.IN...,chrX,153669436,153669453


In [68]:
df_old_share[['a', 'b', 'c', 'id_merge']].to_csv('../tables/2022-11-25_old_650_ND456.bed', header=False, index=False,
                                                 sep='\t')

In [69]:
# end2 为-1 代表老点不存在，
# end2 有值说明，新老点重合，以老点 region 坐标为准进行替换

# 唯一标识mpmat_index
df_bed_to_fix['mpmat_index'] = (
        df_bed_to_fix['chrom'] +
        '_' +
        df_bed_to_fix['start'].astype(str) +
        '_' +
        df_bed_to_fix['end'].astype(str)
)

df_bed_to_fix_part1 = (
    df_bed_to_fix
    .query('end2 == -1')
    [['mpmat_index', 'chrom', 'start', 'end']]
    .copy()
)

df_bed_to_fix_part2 = (
    df_bed_to_fix
    .query('end2 != -1')
    [['mpmat_index', 'chrom2', 'start2', 'end2']]
    .copy()
)

df_bed_to_fix_part1.columns = ['mpmat_index', 'chr_name', 'region_start', 'region_end']
df_bed_to_fix_part2.columns = ['mpmat_index', 'chr_name', 'region_start', 'region_end']

df_bed_fixed_coordinate = pd.concat(
    [df_bed_to_fix_part1, df_bed_to_fix_part2],
    axis=0
)
df_bed_fixed_coordinate

,mpmat_index,chr_name,region_start,region_end
0,chr1_630967_631005,chr1,630967,631005
1,chr1_631016_631037,chr1,631016,631037
2,chr1_631778_631791,chr1,631778,631791
3,chr1_632382_632387,chr1,632382,632387
4,chr1_633067_633147,chr1,633067,633147
...,...,...,...,...
8156,chr12_117022857_117022923,chr12,117022835,117022888
8255,chr13_49219804_49219857,chr13,49219804,49219836
8933,chr16_78770741_78770746,chr16,78770741,78770752
8995,chr17_9576625_9576657,chr17,9576625,9576648


In [70]:
# 发现两个 duplicated 的 mpmat_index，check 一下为什么
# print(df_bed_fixed_coordinate[df_bed_fixed_coordinate.mpmat_index.duplicated()])

print(df_bed_fixed_coordinate.query('mpmat_index=="chr10_22989043_22989070"'))
print(df_bed_fixed_coordinate.query('mpmat_index=="chr17_67963461_67963466"'))

#                  mpmat_index chr_name  region_start  region_end
# 864  chr10_22989043_22989070    chr10      22989049    22989070
# 865  chr10_22989043_22989070    chr10      22989052    22989071
#                   mpmat_index chr_name  region_start  region_end
# 2931  chr17_67963461_67963466    chr17      67963444    67963466
# 2932  chr17_67963461_67963466    chr17      67963458    67963466

# 发现其实就是没定下来最终的导致了有多出来的点，图方便取第一个，注意以后 debug！
# DEBUG

df_bed_fixed_coordinate.drop_duplicates(subset='mpmat_index',
                                        keep='first',
                                        inplace=True)

df_bed_fixed_coordinate.head(2)

                 mpmat_index chr_name  region_start  region_end
947  chr10_22989043_22989070    chr10      22989049    22989070
948  chr10_22989043_22989070    chr10      22989052    22989071
                  mpmat_index chr_name  region_start  region_end
3491  chr17_67963461_67963466    chr17      67963444    67963466
3492  chr17_67963461_67963466    chr17      67963458    67963466


,mpmat_index,chr_name,region_start,region_end
0,chr1_630967_631005,chr1,630967,631005
1,chr1_631016_631037,chr1,631016,631037


In [71]:
df_bed_fixed_coordinate2 = (
    df_bed_fixed_coordinate
    .merge(df_sign_lenient, on=['mpmat_index'], how='left')
    .drop(
        columns=['chr_name_y', 'region_start_y', 'region_end_y']
    )
    .rename(
        columns={
            'chr_name_x': 'chr_name',
            'region_start_x': 'region_start',
            'region_end_x': 'region_end'}
    )
)

sample_names = df_bed_fixed_coordinate2.pop('<sample>')
mpmat_indexes = df_bed_fixed_coordinate2.pop('mpmat_index')
# 利用insert方法插入取出的数据列到指定位置

df_bed_fixed_coordinate2.insert(0, '<sample>', sample_names)
df_bed_fixed_coordinate2.insert(4, 'mpmat_index', mpmat_indexes)

# fix mpmat_index
df_bed_fixed_coordinate2['mpmat_index'] = df_bed_fixed_coordinate2['chr_name'] + '_' + \
                                          df_bed_fixed_coordinate2['region_start'].astype(str) + '_' + \
                                          df_bed_fixed_coordinate2['region_end'].astype(str)

df_bed_fixed_coordinate2

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR,bed_name,strand
0,DetectSeq_ATP8-DddA11_REP-1,chr1,630967,631005,chr1_630967_631005,7,1,6,chr1_630992_GA,2975,4635,0.641855,38,6608,1,780,0.096198,39.389272,0.002532,4.649460,"0,1,2,3,4,5,6,7 35,2,1,0,0,0,0,...",8.677587,10.842847,TestOK,2.511695e-310,5.507753e-305,chr1_630967_631005_highest_chr1...,.
1,DetectSeq_ATP8-DddA11_REP-1,chr1,631016,631037,chr1_631016_631037,6,0,6,chr1_631036_CT,1732,4662,0.371514,33,6236,0,1189,0.083540,37.171837,0.000000,7.087446,"0,1,2,3,4,5,6 33,0,0,0,0,0,0 31...",8.797528,9.773088,TestOK,0.000000e+00,0.000000e+00,chr1_631016_631037_highest_chr1...,.
2,DetectSeq_ATP8-DddA6_REP-1,chr1,631016,631037,chr1_631016_631037,6,0,6,chr1_631036_CT,1546,3564,0.433782,33,4901,0,770,0.083540,31.497951,0.000000,4.948668,"0,1,2,3,4,5,6 33,0,0,0,0,0,0 24...",8.558576,9.254862,TestOK,4.024876e-307,1.029691e-301,chr1_631016_631037_highest_chr1...,.
3,DetectSeq_ATP8-DddA11_REP-1,chr1,631778,631791,chr1_631778_631791,3,0,3,chr1_631791_CT,733,1147,0.639058,40,1483,0,547,0.101261,8.839935,0.000000,3.260583,"0,1,2,3 40,0,0,0 534,278,413,258",6.447891,8.652952,TestOK,2.915445e-214,4.972421e-209,chr1_631778_631791_highest_chr1...,.
4,DetectSeq_ATP8-DddA6_REP-1,chr1,631778,631791,chr1_631778_631791,3,0,3,chr1_631787_CT,731,1055,0.692891,40,1447,0,467,0.101261,9.299640,0.000000,3.001335,"0,1,2,3 40,0,0,0 530,320,528,69",6.521031,8.533426,TestOK,3.072057e-182,6.736538e-177,chr1_631778_631791_highest_chr1...,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12344,DetectSeq_SIRT6-DddA11_REP-2,chr12,117022835,117022888,chr12_117022835_117022888,11,0,11,chr12_117022887_CT,47,96,0.489583,26,158,0,38,0.065819,0.733786,0.000000,0.176480,"0,1,2,3,4,5,6,7,8,9,10,11 25,1,...",3.478776,4.888107,TestOK,2.478798e-12,2.231631e-09,chr12_117022857_117022923_highe...,.
12345,DetectSeq_SIRT6-DddA11_REP-2,chr13,49219804,49219836,chr13_49219804_49219836,15,0,15,chr13_49219811_CT,15,55,0.272727,13,85,0,37,0.032910,0.394758,0.000000,0.171836,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",3.584386,4.401052,TestOK,2.131671e-11,1.643442e-08,chr13_49219804_49219857_highest...,.
12346,DetectSeq_SIRT6-DddA11_REP-2,chr16,78770741,78770752,chr16_78770741_78770752,4,0,4,chr16_78770741_CT,31,44,0.704545,12,49,0,22,0.030378,0.227567,0.000000,0.102173,"0,1,2,3,4 12,0,0,0,0 5,22,19,3,0",2.905182,3.927556,TestOK,7.988221e-07,2.144053e-04,chr16_78770741_78770746_highest...,.
12347,DetectSeq_SIRT6-DddA11_REP-2,chr17,9576625,9576648,chr17_9576625_9576648,12,0,12,chr17_9576625_CT,16,61,0.262295,23,85,0,18,0.058225,0.394758,0.000000,0.083596,"0,1,2,3,4,5,6,7,8,9,10,11,12 20...",2.761264,3.655176,TestOK,1.286750e-05,2.301233e-03,chr17_9576625_9576657_highest_c...,.


In [72]:
# sample 内部无重合
# 若有重合，仔细检查原因
for sample, _df in df_bed_fixed_coordinate2.groupby('<sample>'):
    dup_num = _df.mpmat_index.duplicated().sum()
    print(dup_num)

    if dup_num:
        print(f'sample={sample}\tdup_num={dup_num}')
        print(_df[_df.mpmat_index.duplicated()].iloc[:, 0:6])
        print()
    else:
        pass

0
0
0
1
sample=DetectSeq_JAK2-DddA11_REP-1	dup_num=1
                          <sample> chr_name  region_start  region_end              mpmat_index  region_site_num
11921  DetectSeq_JAK2-DddA11_REP-1    chr12      81451345    81451359  chr12_81451345_81451359                7

0
1
sample=DetectSeq_SIRT6-DddA11_REP-1	dup_num=1
                           <sample> chr_name  region_start  region_end              mpmat_index  region_site_num
11922  DetectSeq_SIRT6-DddA11_REP-1    chr12      81451345    81451359  chr12_81451345_81451359                7

1
sample=DetectSeq_SIRT6-DddA11_REP-2	dup_num=1
                           <sample> chr_name  region_start  region_end              mpmat_index  region_site_num
11923  DetectSeq_SIRT6-DddA11_REP-2    chr12      81451345    81451359  chr12_81451345_81451359                7



In [73]:
# 这个点被 call 到了两次，因为视为了两个点: chr12_81451345_81451359
# 为了后续分析的健壮，将其中之一去除
print(df_bed_fixed_coordinate2.query('mpmat_index=="chr1_9702414_9702449"').iloc[:, 0:7])
print(df_bed_fixed_coordinate2.query('mpmat_index=="chr12_81451345_81451359"').iloc[:, 0:7])
# 去掉重合的，因为mpmat_index一致，故视为一个 region，
# 这个点特殊在 call 到了两个信号的区域，merge 成了一个点，图方便直接保留其中之一
df_bed_fixed_coordinate2 = df_bed_fixed_coordinate2.iloc[-11918:11920].copy()

print(df_bed_fixed_coordinate2.query('mpmat_index=="chr12_81451345_81451359"').iloc[:, 0:7])

df_bed_fixed_coordinate2

                           <sample> chr_name  region_start  region_end           mpmat_index  region_site_num  region_block_site_num
11324   DetectSeq_JAK2-DddA11_REP-1     chr1       9702414     9702449  chr1_9702414_9702449                8                      0
11325   DetectSeq_JAK2-DddA11_REP-2     chr1       9702414     9702449  chr1_9702414_9702449                8                      0
11326  DetectSeq_SIRT6-DddA11_REP-1     chr1       9702414     9702449  chr1_9702414_9702449                8                      0
11327  DetectSeq_SIRT6-DddA11_REP-2     chr1       9702414     9702449  chr1_9702414_9702449                8                      0
                           <sample> chr_name  region_start  region_end              mpmat_index  region_site_num  region_block_site_num
11918   DetectSeq_JAK2-DddA11_REP-1    chr12      81451345    81451359  chr12_81451345_81451359               16                      0
11919  DetectSeq_SIRT6-DddA11_REP-1    chr12      81451345    8

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR,bed_name,strand
431,DetectSeq_ATP8-DddA6_REP-1,chr2,75651730,75651767,chr2_75651730_75651767,14,0,14,chr2_75651767_GA,9,27,0.333333,27,38,0,22,0.068351,0.244220,0.0,0.141391,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",1.837149,4.596991,TestOK,2.221841e-07,6.593459e-04,chr2_75651730_75651767_highest_...,.
432,DetectSeq_ATP8-DddA6_REP-1,chr2,84975098,84975121,chr2_84975098_84975121,10,0,10,chr2_84975120_GA,10,36,0.277778,15,49,0,27,0.037973,0.314915,0.0,0.173525,"0,1,2,3,4,5,6,7,8,9,10 15,0,0,0...",3.051929,4.892447,TestOK,4.238782e-09,1.995855e-05,chr2_84975098_84975121_highest_...,.
433,DetectSeq_ATP8-DddA6_REP-1,chr2,85026836,85026934,chr2_85026836_85026934,20,0,20,chr2_85026934_GA,14,36,0.388889,37,50,0,32,0.093666,0.321342,0.0,0.205659,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",1.778512,5.137560,TestOK,7.464875e-11,4.939012e-07,chr2_85026836_85026934_highest_...,.
434,DetectSeq_ATP8-DddA6_REP-1,chr2,97569473,97569556,chr2_97569473_97569556,17,0,17,chr2_97569548_GA,11,34,0.323529,30,59,0,20,0.075945,0.379184,0.0,0.128537,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",2.319862,4.459488,TestOK,1.053268e-06,2.633154e-03,chr2_97569473_97569556_highest_...,.
435,DetectSeq_ATP8-DddA6_REP-1,chr2,108571354,108571423,chr2_108571354_108571423,19,0,19,chr2_108571401_GA,7,22,0.318182,28,33,0,19,0.070882,0.212086,0.0,0.122110,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",1.581149,4.385487,TestOK,2.277520e-06,5.103252e-03,chr2_108571354_108571423_highes...,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11915,DetectSeq_SIRT6-DddA11_REP-2,chr12,69157217,69157231,chr12_69157217_69157231,10,0,10,chr12_69157217_CT,25,64,0.390625,11,83,0,18,0.027847,0.385470,0.0,0.083596,"0,1,2,3,4,5,6,7,8,9,10 11,0,0,0...",3.791043,3.810104,TestOK,1.286750e-05,2.301233e-03,chr12_69157217_69157238_highest...,.
11916,DetectSeq_JAK2-DddA11_REP-1,chr12,69879529,69879574,chr12_69879529_69879574,3,0,3,chr12_69879574_GA,33,81,0.407407,22,102,0,26,0.055693,0.624660,0.0,0.159227,"0,1,2,3 21,1,0,0 35,41,24,2",3.487496,4.739686,TestOK,1.114112e-08,3.475917e-05,chr12_69879569_69879574_highest...,.
11917,DetectSeq_SIRT6-DddA11_REP-1,chr12,69879529,69879574,chr12_69879529_69879574,3,0,3,chr12_69879574_GA,36,91,0.395604,22,107,0,26,0.055693,0.595055,0.0,0.144593,"0,1,2,3 21,1,0,0 45,36,23,3",3.417447,4.600596,TestOK,1.577425e-08,1.373416e-05,chr12_69879569_69879574_highest...,.
11918,DetectSeq_JAK2-DddA11_REP-1,chr12,81451345,81451359,chr12_81451345_81451359,16,0,16,chr12_81451359_GA,14,32,0.437500,32,58,0,32,0.081008,0.355199,0.0,0.195972,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",2.132483,5.039247,TestOK,9.023829e-11,3.734094e-07,chr12_81451274_81451359_highest...,.


In [74]:
df_pois = df_bed_fixed_coordinate2.copy()
df_pois.head()

# 现在只要是有 nature 当中的 region，都和 nature 的 coordinate
# 和mpmat_index保持一致了，后面如果回溯命名，直接和 v4 的 table merge一下即可

,<sample>,chr_name,region_start,region_end,mpmat_index,region_site_num,region_block_site_num,region_mut_site_num,region_highest_site_index,region_highest_site_mut_num,region_highest_site_cover_num,region_highest_site_mut_ratio,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR,bed_name,strand
431,DetectSeq_ATP8-DddA6_REP-1,chr2,75651730,75651767,chr2_75651730_75651767,14,0,14,chr2_75651767_GA,9,27,0.333333,27,38,0,22,0.068351,0.244220,0.0,0.141391,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",1.837149,4.596991,TestOK,2.221841e-07,6.593459e-04,chr2_75651730_75651767_highest_...,.
432,DetectSeq_ATP8-DddA6_REP-1,chr2,84975098,84975121,chr2_84975098_84975121,10,0,10,chr2_84975120_GA,10,36,0.277778,15,49,0,27,0.037973,0.314915,0.0,0.173525,"0,1,2,3,4,5,6,7,8,9,10 15,0,0,0...",3.051929,4.892447,TestOK,4.238782e-09,1.995855e-05,chr2_84975098_84975121_highest_...,.
433,DetectSeq_ATP8-DddA6_REP-1,chr2,85026836,85026934,chr2_85026836_85026934,20,0,20,chr2_85026934_GA,14,36,0.388889,37,50,0,32,0.093666,0.321342,0.0,0.205659,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",1.778512,5.137560,TestOK,7.464875e-11,4.939012e-07,chr2_85026836_85026934_highest_...,.
434,DetectSeq_ATP8-DddA6_REP-1,chr2,97569473,97569556,chr2_97569473_97569556,17,0,17,chr2_97569548_GA,11,34,0.323529,30,59,0,20,0.075945,0.379184,0.0,0.128537,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",2.319862,4.459488,TestOK,1.053268e-06,2.633154e-03,chr2_97569473_97569556_highest_...,.
435,DetectSeq_ATP8-DddA6_REP-1,chr2,108571354,108571423,chr2_108571354_108571423,19,0,19,chr2_108571401_GA,7,22,0.318182,28,33,0,19,0.070882,0.212086,0.0,0.122110,"0,1,2,3,4,5,6,7,8,9,10,11,12,13...",1.581149,4.385487,TestOK,2.277520e-06,5.103252e-03,chr2_108571354_108571423_highes...,.


In [75]:
df_pois.isna().sum().sum()

0

In [76]:
# df_pois.to_csv('/Users/zhaohuanan/Downloads/2022-11-25_for_home_use.csv.gz', index=None)
df_pois.to_csv('../tables/2023-04-07_for_home_use.csv.gz', index=None)